## Rechtspraak ETL

In [1]:
!pip install mysql.connector

Processing /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175/mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [3]:
CONNECTION = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}:{PORT}/{DATABASE}'

In [4]:
engine = create_engine(CONNECTION, echo=False)#, echo=True)
connection = engine.connect()
query = """SELECT * FROM caselaw.case LIMIT 1;"""
check = pd.read_sql_query(query, con=connection)
check.head()

,id,date,name,description,language,venue,abstract,procedure_type,lodge_date,link,ecli,court_id


---
### Datasets

In [5]:
df_case_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case.csv").replace(np.nan, 'NULL', regex=True)
df_country = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/countries.csv").replace(np.nan, 'NULL', regex=True)
df_li_cases = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legal_intelligence_cases.csv").replace(np.nan, 'NULL', regex=True).drop_duplicates(subset ="ecli")
df_case_opinion = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case_opinion_from_advocate_general.csv").replace(np.nan, 'NULL', regex=True)
df_case_citation = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/caselaw_citations.csv").replace(np.nan, 'NULL', regex=True)
df_legislation_citation = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legislation_citations.csv").replace(np.nan, 'NULL', regex=True)

---
### Normalization based on constrains

In [6]:
#li formating
for i, li_case in df_li_cases.iterrows():
    ecli = li_case.ecli
    new_ecli = ecli.replace('_', ':')
    df_li_cases.at[i, 'ecli'] = new_ecli

li_intersection = list(set(df_li_cases['ecli']).intersection(set(df_case_raw['case_id'])))

#sample for testing
#sample_size = 500000
#uniques = list(df_case_citation['source_ecli'].sample(n=sample_size, random_state=18))
#uniques.extend(li_intersection)

uniques = list(df_case_citation['source_ecli'])
uniques.extend(li_intersection)

df_case = df_case_raw[df_case_raw['case_id'].isin(uniques)]

##1: legislation citations 
df_legislation_citation = df_legislation_citation[df_legislation_citation['source_ecli'].isin(uniques)].reset_index(drop=True)

##2: case citations 
df_case_citation = df_case_citation[df_case_citation['source_ecli'].isin(uniques)].reset_index(drop=True)

##3: case opinions 
df_case_opinion = df_case_opinion[df_case_opinion['case_id'].isin(uniques)].reset_index(drop=True)

##4: li cases 
df_li_cases = df_li_cases[df_li_cases['ecli'].isin(uniques)].reset_index(drop=True)

print('Number of Cases in\nCases: {}\nLegislation citation: {}\nCase citation: {}\nCase opinion: {}\nLI cases: {}'\
      .format(len(df_case), len(df_legislation_citation), len(df_case_citation), len(df_case_opinion), len(df_li_cases)))

Number of Cases in
Cases: 15909
Legislation citation: 18947
Case citation: 758268
Case opinion: 0
LI cases: 617


In [7]:
del uniques

---
### Utils

In [8]:
import string

def clean_strings(column):
  cleaned = []
  for s in column:
    cleaned.append("".join(filter(lambda char: char in string.printable, s)))
  return [i[0:250] for i in cleaned]

In [9]:
def clean_table_sql(table_name):
  engine.execute("""DELETE FROM `{}`;""".format(table_name))
  engine.execute("""ALTER TABLE `{}` AUTO_INCREMENT = 1;""".format(table_name))

In [10]:
def get_parent_ids(table, column_table, df, column_df):
  """DB table, DB column_table, df: pandas df to look at, column_df"""
  read_all_ids = pd.read_sql("""SELECT id, `{}` FROM `{}` """.format(column_table, table), con=connection)
  id_list = []
  for idx, data in enumerate(df[column_df]):
    id_list.append(read_all_ids[read_all_ids[column_table] == data].id.values[0])
  return id_list

---
### Tables Hierarchies

1. Court
2. Case
3. Case Opinion
4. LI Case
5. Subject
6. Case subject
7. Country
8. Case country
9. Case citation
10. Legislation citation
11. Case related decision


In [11]:
# clean_table_sql('case_related_decision')
# clean_table_sql('legislation_citation')
# clean_table_sql('case_citation')
# clean_table_sql('case_country')
# clean_table_sql('country')
# clean_table_sql('case_subject')
# clean_table_sql('subject')
# clean_table_sql('legal_intelligence_case')
# clean_table_sql('case_opinion')
# clean_table_sql('case')
# clean_table_sql('court')

---
### Courts

In [12]:
court = pd.DataFrame()
courts_list = df_case_raw.authority.unique()
courts_list = [i.replace('"','-') for i in courts_list]
courts_list.extend(['Other'])

In [13]:
court['name'] = clean_strings(courts_list)
court.loc[:,'type'] = 'NULL'
court.loc[:,'level'] = 'NULL'
court.loc[:,'country'] = 'NULL'
court.loc[:,'language'] = 'NULL' 
court.loc[:,'jurisdiction'] = 'NULL' 
court.loc[:,'law_area'] = 'NULL'
court.loc[:,'authority_level'] = 'NULL'

In [14]:
court.head(2)

,name,type,level,country,language,jurisdiction,law_area,authority_level
0,Gerechtshof Amsterdam,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,Gerechtshof Arnhem,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [15]:
court.to_sql('court', con=engine, index=False, if_exists='append', chunksize = 10)
del court

---
### Case

In [16]:
map_case = {'date':'date',
            'description':'description',
            'language':'language',
            'venue':'venue',
            'abstract':'abstract',
            'procedure_type':'procedure_type',
            'lodge_date':'lodge_date',
            'alternative_sources':'link',
            'case_id':'ecli'}

case = df_case[map_case.keys()].rename(columns=map_case)
case['name'] = 'NULL'
case['court_id'] = get_parent_ids('court', 'name', df_case, 'authority')
case['date'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in case['date']]
case['lodge_date'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in case['lodge_date']]
case['description'] = clean_strings(case['description'])
case['link'] = clean_strings(case['link'])

In [17]:
case.head(3)

,date,description,language,venue,abstract,procedure_type,lodge_date,link,ecli,name,court_id
43,1913-02-24,NULL,NL,NULL,NULL,NULL,1900-01-01,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:HR:1913:132,NULL,6
60,1913-04-25,NULL,NL,NULL,NULL,NULL,1900-01-01,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:HR:1913:130,NULL,6
63,1913-04-11,NULL,NL,NULL,NULL,NULL,1900-01-01,"\n \n <rdf:list xmlns:rdf=""htt...",ECLI:NL:HR:1913:134,NULL,6


In [18]:
case.to_sql('case', con=engine, index=False, if_exists='append', chunksize = 100)

In [19]:
del map_case
del case

---
### Case opinion advocate general

In [20]:
map_case_opinion = {'date':'date',
                    'case_number':'case_number',
                    'description':'description',
                    'language':'language',
                    'country':'country',
                    'venue':'venue',
                    'abstract':'abstract',
                    'procedure_type':'procedure_type',
                    'authority':'authority',
                    'case_id':'ecli'}
                    
case_opinion = df_case_opinion[map_case_opinion.keys()].rename(columns=map_case_opinion)
case_opinion['date'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in case_opinion['date']]
case_opinion['abstract'] = clean_strings(case_opinion['abstract'])

In [21]:
case_opinion.head(2)

,date,case_number,description,language,country,venue,abstract,procedure_type,authority,ecli


In [22]:
case_opinion.to_sql('case_opinion', con=engine, index=False, if_exists='append', chunksize = 100)
del map_case_opinion
del case_opinion

---
### Legal Intelligence Cases

In [23]:
map_legal_intelligence_case = {
    'ecli':'ecli',
    'Title':'name',
    'date':'date',
    'abstract':'abstract',
    'LawArea':'subject',
    'Url':'link',
    'DisplayTitle':'DisplayTitle',
    'OriginalUrl':'OriginalUrl',
    'Jurisdiction':'Jurisdiction',
    'DocumentType':'DocumentType',
    'case_number':'CaseNumber',
    'PublicationNumber':'PublicationNumber',
    'IssueNumber':'IssueNumber',
    'lodge_date':'lodge_date',
    'DateAdded':'DateAdded',
    'Sources':'Sources',
    'UrlWithAutoLogOnToken':'UrlWithAutoLogOnToken',
    'authority':'court',
    'DisplaySubtitle':'DisplaySubtitle'
}

legal_intelligence_case = df_li_cases[map_legal_intelligence_case.keys()].rename(columns=map_legal_intelligence_case)
legal_intelligence_case['date'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in df_li_cases['date']]
legal_intelligence_case['lodge_date'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in df_li_cases['lodge_date']]
legal_intelligence_case['DateAdded'] = [pd.format_date(i, errors='coerce') if i != 'NULL' else pd.format_date('1900-01-01 00:00:00') for i in df_li_cases['DateAdded']]
legal_intelligence_case['name'] = clean_strings(legal_intelligence_case['name'])
legal_intelligence_case['abstract'] = clean_strings(legal_intelligence_case['abstract'])
legal_intelligence_case['DisplayTitle'] = clean_strings(legal_intelligence_case['DisplayTitle'])
legal_intelligence_case['court'] = clean_strings(legal_intelligence_case['court'])
legal_intelligence_case['DisplaySubtitle'] = clean_strings(legal_intelligence_case['DisplaySubtitle'])

In [24]:
legal_intelligence_case.head(2)

,ecli,name,date,abstract,subject,link,DisplayTitle,OriginalUrl,Jurisdiction,DocumentType,CaseNumber,PublicationNumber,IssueNumber,lodge_date,DateAdded,Sources,UrlWithAutoLogOnToken,court,DisplaySubtitle
0,ECLI:NL:RVS:1996:AP7727,Casus Bij besluit van 12 april 1994 verlenen g...,1970-01-01 00:00:00.019960101,Casus Bij besluit van 12 april 1994 verlenen g...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,Casus Bij besluit van 12 april 1994 verlenen g...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,Rechtspraak,E03951413,NULL,NULL,1970-01-01 00:00:00.019960916,1970-01-01 00:00:00.020110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Raad van State,"Doorduijn, L.F. - Milieurecht Totaal - Module-..."
1,ECLI:NL:RVS:1996:AP7883,M.N. Boeve Casus GS van Noord-Brabant hebben (...,1970-01-01 00:00:00.019960101,M.N. Boeve Casus GS van Noord-Brabant hebben (...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,http://www.legalintelligence.com/documents/496...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,https://www.navigator.nl/#/document/dae9667789...,Nederland,Rechtspraak,E01.94.0417 ECLI:NL:RVS:1996:AP7883,NULL,NULL,1970-01-01 00:00:00.019960708,1970-01-01 00:00:00.020110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,Raad van State,"Boeve, M.N. - Milieurecht Totaal - Module-seri..."


In [25]:
legal_intelligence_case.to_sql('legal_intelligence_case', con=engine, index=False, if_exists='append', chunksize = 10)
del map_legal_intelligence_case
del legal_intelligence_case

---
### Subjects

In [26]:
subjects_as_list = [list(row.split("; ")) for row in df_case.subject]
unique_subjects = \
    set(list(
        pd.core.common\
            .flatten(subjects_as_list)))
subject = pd.DataFrame()
subject['name'] = clean_strings(list(sorted(unique_subjects)))
subject.loc[:,'standard_name'] = 'NULL'

In [27]:
subject.head(2)

,name,standard_name
0,Ambtenarenrecht,NULL
1,Belastingrecht,NULL


In [28]:
subject.to_sql('subject', con=engine, index=False, if_exists='append', chunksize = 50)
del subject

---
### Case - Subject

In [29]:
df_subjects_case = df_case[['subject','case_id']]
df_subjects_case.loc[:,'subject'] = subjects_as_list
df_subjects_case = df_subjects_case.explode('subject')

parents_ids_subjects = get_parent_ids('subject', 'name', df_subjects_case, 'subject')
parents_ids_cases = get_parent_ids('case', 'ecli', df_subjects_case, 'case_id')
case_subject = pd.DataFrame({'case_id':parents_ids_cases,
                             'subject_id':parents_ids_subjects})
#case_subject.sort_values(by='case_id').tail(50) #quality check
#[(type(i), type(j)) for i,j in zip(case_subject.subject_id,case_subject.case_id)]

In [30]:
case_subject.tail(3)

,case_id,subject_id
16109,15907,5
16110,15908,5
16111,15909,3


In [31]:
case_subject.to_sql('case_subject', con=engine, index=False, if_exists='append', chunksize = 100)
del df_subjects_case
del case_subject

---
### Countries

In [32]:
df_country.loc[:,'language'] = 'NULL'
df_country.loc[:,'eea'] = 0
country = df_country[['country_id','name','language','flag','eu','eea']]\
  .rename(columns={'country_id':'id'})

In [33]:
country.head(3)

,id,name,language,flag,eu,eea
0,AT,Austria,NULL,https://www.countryflags.io/at/flat/64.png,1,0
1,BE,Belgium,NULL,https://www.countryflags.io/be/flat/64.png,1,0
2,BG,Bulgaria,NULL,https://www.countryflags.io/bg/flat/64.png,1,0


In [34]:
country.to_sql('country', con=engine, index=False, if_exists='append', chunksize = 50)
del country

---
### Case - Country

In [35]:
# df_country_case = df_case[['case_id','country']]
# #df_country_case.loc[:,'country'] = as_list
# df_country_case = df_country_case.explode('country')
# len(df_country_case)

In [36]:
# case_country.to_sql('case_country', con=engine, index=False, if_exists='append', chunksize = 100)

In [37]:
# del df_country_case
# del case_country

---
### Case law citation

In [38]:
df_case_citation['case_id'] = get_parent_ids('case', 'ecli', df_case_citation, 'source_ecli')

map_case_citation = {
    'source_ecli':'source_ecli',
    'source_paragraph':'source_paragraph',
    'target_ecli':'target_ecli',
    'target_paragraph':'target_paragraph',
    'case_id':'case_id'
}
case_citation = df_case_citation[map_case_citation.keys()].rename(columns=map_case_citation)

In [39]:
case_citation.head(2)

,source_ecli,source_paragraph,target_ecli,target_paragraph,case_id
0,ECLI:NL:HR:1913:13,NULL,ECLI:NL:HR:1913:129,NULL,5
1,ECLI:NL:HR:1913:130,NULL,ECLI:NL:HR:1913:129,NULL,2


In [40]:
case_citation.to_sql('case_citation', con=engine, index=False, if_exists='append', chunksize = 100)
del case_citation
#del tuples

---
### Legislation citation

In [41]:
parent_ids = get_parent_ids('case', 'ecli', df_legislation_citation, 'source_ecli')
df_legislation_citation['case_id'] = parent_ids
df_legislation_citation.loc[:,'target_name'] = 'NULL'
df_legislation_citation.loc[:,'target_sourcename'] = 'NULL'

map_legislation_citation = {
    'source_ecli':'source_ecli',
    'source_paragraph':'source_paragraph',
    'target_article':'target_id',
    'target_article_paragraph':'target_paragraph',
    'target_name':'target_name',
    'target_sourcename':'target_sourcename',
    'target_article_webpage':'target_link',
    'case_id':'case_id'
}
legislation_citation = df_legislation_citation[map_legislation_citation.keys()].rename(columns=map_legislation_citation)

In [42]:
legislation_citation.head(3)

,source_ecli,source_paragraph,target_id,target_paragraph,target_name,target_sourcename,target_link,case_id
0,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2016-...,8354
1,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2009-...,8354
2,ECLI:NL:GHAMS:1962:AX8440,NULL,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,NULL,NULL,NULL,http://wetten.overheid.nl/id/BWBR0005537/2018-...,8354


In [43]:
legislation_citation.to_sql('legislation_citation', con=engine, index=False, if_exists='append', chunksize = 100)

In [44]:
del legislation_citation

---
### Case related decision

In [45]:
df_case_related = df_case[['case_id','related_cases']]\
    .rename(columns = {'case_id':'source_ecli', 
                       'related_cases': 'referencing_case_ecli'})
df_case_related['case_id'] = get_parent_ids('case', 'ecli', df_case_related, 'source_ecli')
df_case_related.loc[:,'referenced_case_ecli'] = 'NULL'

map_case_related_decision = {
    'source_ecli':'source_ecli',
    'referencing_case_ecli':'referencing_case_ecli',
    'referenced_case_ecli':'referenced_case_ecli',
    'case_id':'case_id'
    }

case_related_decision = df_case_related[map_case_related_decision.keys()].rename(columns=map_case_related_decision)

In [46]:
case_related_decision.head(3)

,source_ecli,referencing_case_ecli,referenced_case_ecli,case_id
43,ECLI:NL:HR:1913:132,NULL,NULL,1
60,ECLI:NL:HR:1913:130,NULL,NULL,2
63,ECLI:NL:HR:1913:134,NULL,NULL,3


In [47]:
case_related_decision.to_sql('case_related_decision', con=engine, index=False, if_exists='append', chunksize = 100)
del case_related_decision